In [2]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import random
import glob
import umap
import seaborn as sns
import matplotlib.pyplot as plt
# from ggplot import *


from ruffus import *
import sys, os, h5py, random, tempfile, scipy, time,copy
import cmapPy.pandasGEXpress.parse_gctx as parse_gctx
import cmapPy.pandasGEXpress.parse_gct as parse_gct
import pipeline_support as PS
from sklearn.decomposition import PCA
from scipy import stats
from matplotlib.pyplot import imshow
from sklearn.preprocessing import MinMaxScaler

from collections import Counter
#from tensorflow_examples.models.pix2pix import pix2pix
import tensorflow as tf
from tensorflow import keras
from IPython.display import clear_output
from tensorflow_gan.python.losses import losses_impl


from numpy.random import seed
randomState = 123
seed(randomState)

In [1]:
ARCHS4_filename = "../data/ARCHS4/human_matrix_v9.h5"
l1000_filename = "../data/L1000/GSE92742_Broad_LINCS_Level3_INF_mlr12k_n1319138x12328.gctx"
l1000_geneinfo_filename = "../data/L1000/GSE92742_Broad_LINCS_gene_info.txt"
gtex_rnaseq_filename = "../data/GTEx/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_reads.gct"
gtex_l1000_filename = "../data/GTEx/DS_GTEX_L1000_n3176x12320.gctx"
# gene_id_to_symbol_filename = "../data/GTEx/Gene_ID_to_Symbol_GRCh37.txt"
# gene_transcript_id_to_hgnc_gene_filename = "../data/GTEx/gencode.v19.metadata.HGNC"
gtex_geneinfo_filename = "../data/GTEx/GSE92743_Broad_GTEx_gene_info.txt"

In [3]:
l1000_all_gene_list = "../data/L1000/all_gene_list.txt"
l1000_landmark_gene_list = "../data/L1000/landmark_gene_list.txt"

archs4_all_gene_list = "../data/ARCHS4/all_gene_list.txt"

gtex_l1000_all_gene_list = "../data/GTEx/l1000_all_gene_list.txt"
gtex_l1000_landmark_gene_list = "../data/GTEx/l1000_landmark_gene_list.txt"

gtex_rnaseq_all_gene_list = "../data/GTEx/rnaseq_all_gene_list.txt"

# Load L1000

In [3]:
print('Loading L1000 data.....')
l1000_data = parse_gctx.parse(l1000_filename,convert_neg_666=True).data_df

Loading L1000 data.....


In [5]:
gene_info = pd.read_csv(l1000_geneinfo_filename,header = 0, sep = '\t')

In [6]:
# create a probe_id to gene name dictionary 
gene_dict = dict(zip([str(x) for x in gene_info['pr_gene_id']], gene_info['pr_gene_symbol']))

# label rows with gene names 
l1000_data.index = [gene_dict[x] for x in l1000_data.index.values]


In [8]:
# save all genes
with open(l1000_all_gene_list, "w") as f:
    f.write("\n".join(l1000_data.index.tolist()))

In [9]:
# filter landmark genes
landmark_gene_info = gene_info[gene_info["pr_is_lm"] == 1]
l1000_data_landmark_genes = l1000_data.loc[landmark_gene_info["pr_gene_symbol"], :]

In [11]:
# Save L1000 genes to a txt file
with open(l1000_landmark_gene_list, "w") as f:
    f.write("\n".join(l1000_data_landmark_genes.index.tolist()))

# ARCHS4

In [12]:
# Import ARCHS4 RNA-seq samples 
print('Processing RNA-seq data.....')
h5 = h5py.File(ARCHS4_filename, 'r')
data_file = h5['data'] 
expression = data_file['expression']
genes = [x for x in h5['meta']['genes']['genes']]

Processing RNA-seq data.....


In [16]:
with open(archs4_all_gene_list, "w") as f:
    f.write("\n".join(genes))

# GTEx

## GTEx L1000

In [20]:
gtex_gene_info = pd.read_csv(gtex_geneinfo_filename,header = 0, sep = '\t')
gtex_landmark_genes = gtex_gene_info.loc[gtex_gene_info["pr_is_lm"]==1, "pr_gene_symbol"].tolist()

In [21]:
# GTEx L1000 data
print('Loading GTEx L1000 data.....')
gtex_l1000_data = parse_gctx.parse(gtex_l1000_filename,convert_neg_666=True).data_df

# create a probe_id to gene name dictionary 
gtex_gene_dict = dict(zip([str(x) for x in gtex_gene_info['pr_gene_id']], gtex_gene_info['pr_gene_symbol']))

# label rows with gene names 
gtex_l1000_data.index = [gtex_gene_dict[x] for x in gtex_l1000_data.index.values]


gtex_l1000_data_landmark = gtex_l1000_data.loc[gtex_l1000_data.index.isin(gtex_landmark_genes), :]

Loading GTEx L1000 data.....


In [22]:
with open(gtex_l1000_all_gene_list, "w") as f:
    f.write("\n".join(gtex_l1000_data.index.tolist()))

In [29]:
with open(gtex_l1000_landmark_gene_list, "w") as f:
    f.write("\n".join(gtex_l1000_data_landmark.index.tolist()))

## GTEx RNA-seq

In [32]:
with open(gtex_rnaseq_filename, "r") as fr:
    with open(gtex_rnaseq_all_gene_list, "w") as f:
        lines = fr.readlines()
        for i in range(3, len(lines)):
            f.write(lines[i].split("\t")[1])
            f.write("\n")
            f.flush()

